## SUBCIRCUIT SYMMETRY (GENRE)
* Recurrent Network with three Populations: two E populations and one I population
* Percentage of two E populations
* Parameters (new)
    * ppercent Esub -- percentage of neurons in each E population out of the total number of E neurons, summed to 1
    * coeffetaEsub -- degree of symmetry, 3 items 0 $\eta_{E_1}$, 1 $\eta_{E_1E_2}$, 2 $\eta_{E_2}$
    * signetaEsub -- signs of symmetry, 1 -- symmetry -1 -- anti-SYMMETRY

    * coeffetaTotal -- degree of symmetry, 3 items 0 $\eta_{E_1I}$, 1 $\eta_{E_2I}$, and 2 $\eta_{I}$
    * signetaTotal -- signs of symmetry

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

## help functions

In [2]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


import math
import cmath
def ThreeSquare(x):
    if x.imag == 0:
        #real
        m = x.real
        ans = -math.pow(-m,1/3) if m<0 else math.pow(m,1/3)
    else:
        #complex
        ans = x**(1/3)
    return ans

# round to num
def RoundAns(x,num):
    if x.imag == 0:
        m = x.real
        ans = round(m,num)
    else:
        m = round(x.real,num)
        n = round(x.imag,num)
        ans = complex(m,n)
    return ans
    
def Cubic(args):
    a,b,c,d = args
    p = c/a-b**2/(3*a**2)
    q = d/a+2*b**3/(27*a**3)-b*c/(3*a**2)
    w = complex(-0.5,(3**0.5)/2)
    ww = complex(-0.5,-(3**0.5)/2)
    A = cmath.sqrt((q/2)**2+(p/3)**3)
    B = ThreeSquare(-q/2+A)
    C = ThreeSquare(-q/2-A)
    y1 = B+C
    y2 = w*B+ww*C
    y3 = ww*B+w*C
    D = b/(3*a)
    roots=[RoundAns(y1-D,6),RoundAns(y2-D,6),RoundAns(y3-D,6)]
    return roots

## change E and I as well

In [3]:
'''
WITH SYMMETRY
'''
# generate mean matrix
nrank=1
Nt=np.array([500,500])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
'''
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
'''
nrank,ntrial,neta,ngavg=1,20,10,10

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' 
---------- method 2 ------------------ 
'''
''' ## Three \bar{J} cases '''
JI,JE,a,b=3.5,1.0,0,0
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference '''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

'''
##>>>>>>>>>>> g0!=0, one example >>>>>>>>
'''
gaverage=1.0
# X0=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
X0=np.zeros((NE*2,NE*2))
X0[:NE,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[:NE,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
Xsym=X0.copy()
XsymT=Xsym.copy().T
J0=Am.copy()+X0.copy()
## >>>>>> calculate the original \bar{m} and \bar{n} 
eigvJ0,eigrvecJ0=la.eig(J0)
inveigrvecJ0=la.inv(eigrvecJ0)
meig0 = np.squeeze(eigrvecJ0[:,:].copy())
neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
neig0=neig0.copy().T
## test neig0*meig0=1
print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
for i in range(nrank):
    neig0[:,i]*=eigvJ0[i]
for j in range(1,N):
    neig0[:,j]*=eigvJ0[j]
## >>>>> eigendecomposition 
leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
''' The first '''
reig0=meig0.copy()
normval=np.sum(reig0*neig0.copy(),axis=0)
normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
leig0=neig0.copy()/normval.copy()
''' calculate s0'''
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvJ0[i+1]
    Z[i,i]=1/(b1-eigvJ0[0])
S0=reig0[:,1:]@Z@leig0[:,1:].T
print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
print('eigenvalues:',eigvJ0[:nrank])
'''## for reference '''
x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
tildex,tildey=x0[:,0].copy(),y0[:,0].copy()
## make sure the E in x is positive, if negative then all x and y should change sign
if (np.mean(tildey[:NE])*yAm[0,0])<0:
    tildex*=(-1)
    tildey*=(-1)
x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
## >>>> renormalization, mu_l and mu_r are unchanged
x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
'''
##>>>>>>>>>>> end >>>>>>>>
'''

'''
Network Setting for Iterating Following
'''
## heterogeneous random gain
''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 
# coeffetaEsub=np.ones(3)
coeffetaEsub=np.array([0.6,0.0,0.6])
# coeffetaEsub[0]=1.0
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)
# signetaTotal[:]=-1

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]

'''
Recording Variables
'''
# original data
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2),dtype=complex),np.zeros((ngavg,ntrial,NE*2),dtype=complex)
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2),dtype=complex),np.zeros((ngavg,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
x0series,y0series=np.zeros((ntrial,NE*2,1)),np.zeros((ntrial,NE*2,1))
''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)
''' Iterative Processing '''
etaseries=np.linspace(0.0,1.0,neta)
for iktrial in range(ntrial):
    '''
    ##>>>>>>>>>>> g0!=0, >>>>>>>>
    '''
    Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    XsymT=Xsym.copy().T
    '''
    ############## >>>>>>>>>>>> ERROR ...... >>>>>>>>>>>>>>>>>>
    '''
    X0 = Xsym.copy()  
    ##### >>>>>>>>>>>> Adding this >>>>>>>>>>>
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(1,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()
    ''' calculate s0'''
    Z = np.zeros((NE*2-1,NE*2-1))
    for i in range(NE*2-1):
        b1=eigvJ0[i+1]
        Z[i,i]=1/(b1-eigvJ0[0])
    S0=reig0[:,1:]@Z@leig0[:,1:].T
    print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
    print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
    print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    tildex,tildey=np.reshape(reig0[:,0].copy(),(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    # ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    # tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    ## make sure the E in x is positive, if negative then all x and y should change sign
    if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
    x0series[iktrial,:,:],y0series[iktrial,:,:]=x0.copy(),y0.copy()

    for idxeta,eta in enumerate(etaseries[:]):
        etaset=eta*np.ones(6)#np.zeros(3)#eta*np.ones(3)
        for i in range(3):
            etaset[i]=etaset[i]*coeffetaEsub[i]
            etaset[i+3]=etaset[i+3]*coeffetaTotal[i]
        print('Total Symmetry:',etaset)
        Xinit = Xsym.copy()

        ### >>>>>>>>.Subcircuit Exc sym >>>>>>>>>.
        ## remaining
        hNE=int(ppercentEsub[0]*NE) 
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:hNE,:hNE]=signetaEsub[0]*aamp*XsymT[:hNE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,:hNE].copy()
        
        ## >>>>>>. esub middle
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:hNE,hNE:NE]=signetaEsub[1]*aamp*XsymT[:hNE,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,hNE:NE].copy()
        Xinit[hNE:NE,:hNE]=signetaEsub[1]*aamp*XsymT[hNE:NE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,:hNE].copy()
        ## >>>>>> E2
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[hNE:NE,hNE:NE]=signetaEsub[2]*aamp*XsymT[hNE:NE,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,hNE:NE].copy()
        ### >>>>>>> Total >>>>>
        # E1I-B
        hNE = int(ppercentEsub[0]*NE) 
        asqr=(1-np.sqrt(1-(etaset[3])**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[NE:,:hNE]=signetaTotal[0]*aamp*XsymT[NE:,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:hNE].copy()
        Xinit[:hNE,NE:]=signetaTotal[0]*aamp*XsymT[:hNE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,NE:].copy()
        # E2I-B
        asqr=(1-np.sqrt(1-(etaset[4])**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[NE:,hNE:NE]=signetaTotal[1]*aamp*XsymT[NE:,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,hNE:NE].copy()
        Xinit[hNE:NE,NE:]=signetaTotal[1]*aamp*XsymT[hNE:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,NE:].copy()

        ## II ##
        asqr=(1-np.sqrt(1-etaset[5]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signetaTotal[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        
        X=Xinit.copy()
        # incase heterogeneous
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('compare degree of symmetry:',symnumfull,'--num; ')#,etaset*signeta,'--setting')
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        print('test normal 4:',np.sum(neig[:,0]*meig[:,0]))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        '''
        Sort Eigenvalue in ascending 
        '''
        eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
        idxsort=np.argsort(eigenvalue_Rsort)
        eigenvalue_sort = np.squeeze(eigvJ.copy())
        # eigvalseries[idxeta,iktrial,:] = eigenvalue_sort[idxsort]
        eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        meig = meig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        neig = neig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        Reig = meig.copy()
        Leig = neig.copy()
        normval=np.sum(Reig*Leig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=Leig.copy()/normval.copy()  # note that here we change from neig --> Leig, otherwise, if we use copy, there'll be some sort of mistakes
        
        '''
        normalization 0 (reference)
        '''
        tildex,tildey=np.reshape(Reig[:,0].copy(),(2*NE,1)),np.reshape(Leig[:,0].copy(),(NE*2,1))
        ## make sure the E in x is positive, if negative then all x and y should change sign
        if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey
        # if (np.mean(ynorm0[:NE])*yAm[0,0])<0:
        #     xnorm0*=(-1)
        #     ynorm0*=(-1)
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:],Leigvecseries[idxeta,iktrial,:]=xnorm0[:,0].copy(),ynorm0[:,0].copy()

        '''  RECORDING STATES '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE]),np.mean(Reigvecseries[idxeta,iktrial,NE:])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE]),np.std(Reigvecseries[idxeta,iktrial,NE:])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE]),np.mean(Leigvecseries[idxeta,iktrial,NE:])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE]),np.std(Leigvecseries[idxeta,iktrial,NE:])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:])
        reig,leig=np.reshape(tildeReig[:].copy(),(NE*2,1)),np.reshape(tildeLeig[:].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE]),np.mean(reig[NE:])
        lEceneig,lIceneig = np.mean(leig[:NE]),np.mean(leig[NE:])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE],rrandeig[NE:]=rrandeig[:NE]-rEceneig,rrandeig[NE:]-rIceneig
        lrandeig[:NE],lrandeig[NE:]=lrandeig[:NE]-lEceneig,lrandeig[NE:]-lIceneig
        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE],(NE,1)).T@np.reshape(lrandeig[:NE],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:],(NE,1)).T@np.reshape(lrandeig[NE:],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        ''' record the random derivative '''
        if idxeta == 0:
            print('recording!!!')
            ## log: there is a bug, X is the total random xconn, including symmetry
            gYbasexee[iktrial,:,:]=Xsym.copy()

9399e-15j)
Total Symmetry: [0.33333333 0.         0.33333333 0.55555556 0.55555556 0.55555556]
compare degree of symmetry: 0.45867087605644136 --num; 
test normal 4: (1.0000000000000155-9.443031052110607e-16j)
Total Symmetry: [0.4        0.         0.4        0.66666667 0.66666667 0.66666667]
compare degree of symmetry: 0.5502332897964916 --num; 
test normal 4: (0.9999999999999888-2.674847117885073e-14j)
Total Symmetry: [0.46666667 0.         0.46666667 0.77777778 0.77777778 0.77777778]
compare degree of symmetry: 0.6417963372639667 --num; 
test normal 4: (1.0000000000000047-1.8389800168451372e-14j)
Total Symmetry: [0.53333333 0.         0.53333333 0.88888889 0.88888889 0.88888889]
compare degree of symmetry: 0.7333608435286476 --num; 
test normal 4: (1.000000000000002+1.5906160608620283e-15j)
Total Symmetry: [0.6 0.  0.6 1.  1.  1. ]
compare degree of symmetry: 0.824935277011973 --num; 
test normal 4: (1.0000000000000009+1.4529491658467738e-14j)
test normal: (1.000000000000015+2.37325

* recording 

In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationSubcircuitHetereta_data',x0series=x0series,y0series=y0series,xAm=xAm,yAm=yAm,Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,etaseries=etaseries,eigvalseries=eigvalseries[:,:,0],Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,gYbasexee=gYbasexee,coeffetaEsub=coeffetaEsub,coeffetaTotal=coeffetaTotal,signetaEsub=signetaEsub,signetaTotal=signetaTotal,ppercentEsub=ppercentEsub)

* eigenvalue spectrum

In [4]:

'''
with symmetry, subcircuit
eigenvalue spectrum, one trial
'''
idxgavg,idxtrial=9,8 # 6,3,0
figtspt,axtspt=plt.subplots(figsize=(6,3))
cm='bgc'
idxc=0
for idxtrial in range(ntrial):
    axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxc,s=20,c=cm[idxc],marker='^',alpha=0.5) # >>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,:,1]),np.imag(Beigvseries[idxgavg,:,1])-0.2*idxc,s=20,c='r',marker='^',alpha=0.5) # >>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
axtspt.xaxis.set_ticks_position('bottom')
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverage
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
eta=etaseries[idxgavg]
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(-longaxis*np.ones(2), np.array([-1.0,1.0]), color="gray", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-2.5,3.5,2)
xlims = [-3.5,3.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
# axtspt.set_xlim(xlims)
# axtspt.set_ylim(ylims)
# axtspt.set_xticks(xticks)
# axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

1.0 1.0 1.0 1.0
0.9999999999999999
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


* loading activity patterns

In [5]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-0.4,0.45,2)
xlims = [-0.4,0.45]
yticks = np.linspace(-0.4,0.45,2)
ylims = [-0.4,0.45]
# xticks = np.linspace(-0.2,0.25,2)
# xlims = [-0.2,0.25]
# yticks = np.linspace(-0.2,0.25,2)
# ylims = [-0.2,0.25]
# xticks = np.linspace(-0.15,0.10,2)
# xlims = [-0.15,0.10]
# yticks = np.linspace(-0.06,0.01,2)
# ylims = [-0.06,0.01]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=9#16#
idxeta=9#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
hNE = int(ppercentEsub[0]*NE)
'''loading vector changing'''
meanmEs,meanmEn,meanmI,meannEs,meannEn,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEsvec,mEnvec,mIvec,nEsvec,nEnvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:hNE]),np.squeeze(Reigvecseries[idxeta,:,hNE:NE]),\
np.squeeze(Reigvecseries[idxeta,:,NE:]),np.squeeze(Leigvecseries[idxeta,:,:hNE]),np.squeeze(Leigvecseries[idxeta,:,hNE:NE]),np.squeeze(Leigvecseries[idxeta,:,NE:])
mEsvec,mEnvec,mIvec,nEsvec,nEnvec,nIvec=mEsvec.flatten(),mEnvec.flatten(),mIvec.flatten(),nEsvec.flatten(),nEnvec.flatten(),nIvec.flatten()
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEsvec),np.std(mEsvec)
    varmE = mEsvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEsvec[idxwhere]=meanmEtotal
    meanmEs[irank]=np.mean(mEsvec)

    meanmEtotal,stdmEtotal = np.mean(mEnvec),np.std(mEnvec)
    varmE = mEnvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEnvec[idxwhere]=meanmEtotal
    meanmEn[irank]=np.mean(mEnvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEsvec),np.std(nEsvec)
    varnE = nEsvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEsvec[idxwhere]=meannEtotal
    meannEs[irank]=np.mean(nEsvec)

    meannEtotal,stdnEtotal = np.mean(nEnvec),np.std(nEnvec)
    varnE = nEnvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEnvec[idxwhere]=meannEtotal
    meannEn[irank]=np.mean(nEnvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseEs,noiseEn,noiseI = np.zeros((hNE*ntrial,2,nrank)),np.zeros(((NE-hNE)*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecEs,dirvecEn,dirvecI=np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseEs[:,0,irank],noiseEs[:,1,irank]= nEsvec-meannEs[irank],mEsvec-meanmEs[irank]
    noiseEn[:,0,irank],noiseEn[:,1,irank]= nEnvec-meannEn[irank],mEnvec-meanmEn[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirEs,covdirEn,covdirI=np.squeeze(noiseEs[:,:,0]).T@np.squeeze(noiseEs[:,:,0]),np.squeeze(noiseEn[:,:,0]).T@np.squeeze(noiseEn[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecEs=la.eig(covdirEs)
_,dirvecEn=la.eig(covdirEn)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEsvec,mEsvec,s=5.0,c='r',alpha=alphaval)#cm[0],alpha=alphaval)
ax.scatter(nEnvec,mEnvec,s=5.0,c='orange',alpha=alphaval)#cm[0],alpha=alphaval)

ax.plot([meannEn[0],meannEn[0]+dirvecEn[0,0]],[meanmEn[0],meanmEn[0]+dirvecEn[1,0]],color='gray',linestyle='--',linewidth=1.5)
ax.plot([meannEn[0],meannEn[0]+dirvecEn[0,1]],[meanmEn[0],meanmEn[0]+dirvecEn[1,1]],color='gray',linestyle='--',linewidth=1.5)
ax.plot([meannEs[0],meannEs[0]+dirvecEs[0,0]],[meanmEs[0],meanmEs[0]+dirvecEs[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannEs[0],meannEs[0]+dirvecEs[0,1]],[meanmEs[0],meanmEs[0]+dirvecEs[1,1]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEsvec),np.real(mEsvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nEnvec),np.real(mEnvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')
print(etaseries[idxeta])

500
(2, 2)
(2, 2)
(2, 2)
1.0
<ipython-input-5-8772d2b1743c>:44: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmEs[irank]=np.mean(mEsvec)
<ipython-input-5-8772d2b1743c>:50: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmEn[irank]=np.mean(mEnvec)
<ipython-input-5-8772d2b1743c>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-5-8772d2b1743c>:65: ComplexWarning: Casting complex values to real discards the imaginary part
  meannEs[irank]=np.mean(nEsvec)
<ipython-input-5-8772d2b1743c>:71: ComplexWarning: Casting complex values to real discards the imaginary part
  meannEn[irank]=np.mean(nEnvec)
<ipython-input-5-8772d2b1743c>:78: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-5-8772d2b1743c>:85: ComplexWarning: Casting complex values to real discards the imaginary part
  noise

* using determinant lemma, predict the eigenvalue outlier

In [8]:

'''
determinant lemma, odd terms 
even order terms should be maitained
combination

!!! In sparse condition
'''
from scipy.special import comb, perm
gaverage
yticks = np.linspace(-3.40,-2.0,2)
ylims = [-3.4,-2.0]
xticks = np.linspace(-0.1,1.1,2)
xlims = [-0.1,1.1]

nAm=np.reshape(yAm.copy()*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
lambda_theo2 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:,0]),axis=1)
nAmsparse,mAmsparse = nAm.copy(),mAm.copy()
for idxeta,eta in enumerate(etaseries):
    etaE1,etaE12B,etaE2,etaE1IB,etaE2IB,etaI=eta*coeffetaEsub[0],eta*coeffetaEsub[1],eta*coeffetaEsub[2],eta*coeffetaTotal[0],eta*coeffetaTotal[1],eta*coeffetaTotal[2]
    numE1= int(NE*ppercentEsub[0])
    numE2=NE-numE1
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXTE1,bXXTE2,bXXTII=np.eye(numE1),np.eye(numE2),np.eye(NE)
    bXXTE1*=(etaE1*signetaEsub[0]*ppercentEsub[0]+etaE12B*signetaEsub[1]*ppercentEsub[1]+etaE1IB*signetaTotal[0])/4
    bXXTE2*=(etaE12B*signetaEsub[1]*ppercentEsub[0]+etaE2*signetaEsub[2]*ppercentEsub[1]+etaE2IB*signetaTotal[1])/4
    bXXTII*=(etaE1IB*signetaTotal[0]*ppercentEsub[0]+etaE2IB*ppercentEsub[1]*signetaTotal[1]+   etaI*signetaTotal[2])/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    moment2=np.zeros((NE*2,NE*2))
    moment2[:numE1,:numE1]=(bXXTE1)
    moment2[numE1:NE,numE1:NE]=(bXXTE2)
    moment2[NE:,NE:]=bXXTII
    print(np.shape(moment2))
    B2=np.squeeze(C*(nAm.T@(moment2)@mAm)*gaverage**2)
    coeff2 = [1,-eigvAm[0],0,-B2]
    print(coeff2)
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    print(lambda_test2[0])
# fig,ax=plt.subplots(figsize=(5,5))  
# ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
# # ax.plot(lambda_num,lambda_theo,'b')
# ax.plot(lambda_num,lambda_num,'k',label='real')
# # ax.set_xlim(xlims)
# # ax.set_ylim(ylims)
# # ax.set_xticks(xticks)
# # ax.set_yticks(yticks)
# # ax.set_aspect('equal')
# ax.legend()

fig,ax=plt.subplots(figsize=(5,3))  
ax.plot(etaseries,lambda_theo2,color='b',linewidth=1.5,label='theo')
ax.plot(etaseries,lambda_num,color='b',linewidth=1.5,linestyle='--',label='num')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')
ax.legend()

2.0
(1000, 1000)
[1, (2.500000000000015-0j), 0, -0.0]
(-2.500000000000015+0j)
2.0
(1000, 1000)
[1, (2.500000000000015-0j), 0, 0.3166666666666681]
(-2.548747108321872+2.220446049250313e-16j)
2.0
(1000, 1000)
[1, (2.500000000000015-0j), 0, 0.6333333333333362]
(-2.594113999830289+5.551115123125783e-17j)
2.0
(1000, 1000)
[1, (2.500000000000015-0j), 0, 0.9500000000000068]
(-2.6366525665114864-4.440892098500626e-16j)
2.0
(1000, 1000)
[1, (2.500000000000015-0j), 0, 1.2666666666666724]
(-2.6767813860608056+0j)
2.0
(1000, 1000)
[1, (2.500000000000015-0j), 0, 1.5833333333333401]
(-2.7148266462534085+1.1102230246251565e-16j)
2.0
(1000, 1000)
[1, (2.500000000000015-0j), 0, 1.9000000000000137]
(-2.7510482439898585-1.1102230246251565e-16j)
2.0
(1000, 1000)
[1, (2.500000000000015-0j), 0, 2.21666666666668]
(-2.7856571304197386-5.551115123125783e-16j)
2.0
(1000, 1000)
[1, (2.500000000000015-0j), 0, 2.5333333333333448]
(-2.8188272391416547-3.3306690738754696e-16j)
2.0
(1000, 1000)
[1, (2.500000000000015

In [9]:

'''
determinant lemma, odd terms 
even order terms should be maitained
combination

### another solutor
'''
import numpy.polynomial.polynomial as poly
from scipy.special import comb, perm
coeffB4 = np.zeros(neta)
envelopeRe = np.zeros(neta)

yticks = np.linspace(-2.50,1.5,2)
ylims = [-2.6,1.6]
xticks = np.linspace(-0.,1.,2)
xlims = [-0.1,1.1]

nAm=np.reshape(yAm.copy()*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
lambda_theo2 = np.zeros((neta,3),dtype=complex)
nAmsparse,mAmsparse = nAm.copy(),mAm.copy()
for idxeta,eta in enumerate(etaseries[:]):
    etaE1,etaE12B,etaE2=eta*coeffetaEsub[0],eta*coeffetaEsub[1],eta*coeffetaEsub[2]
    etaE1IB,etaE2IB,etaI=eta*coeffetaTotal[0],eta*coeffetaTotal[1],eta*coeffetaTotal[2]
    numE1= int(NE*ppercentEsub[0])
    numE2=NE-numE1
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXTE1,bXXTE2,bXXTII=np.eye(numE1),np.eye(numE2),np.eye(NE)
    bXXTE1*=(etaE1*signetaEsub[0]*ppercentEsub[0]+etaE12B*signetaEsub[1]*ppercentEsub[1]+etaE1IB*signetaTotal[0])/4
    bXXTE2*=(etaE12B*signetaEsub[1]*ppercentEsub[0]+etaE2*signetaEsub[2]*ppercentEsub[1]+etaE2IB*signetaTotal[1])/4
    bXXTII*=(etaE1IB*signetaTotal[0]*ppercentEsub[0]+etaE2IB*ppercentEsub[1]*signetaTotal[1]+   etaI*signetaTotal[2])/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    moment2=np.zeros((NE*2,NE*2))
    moment2[:numE1,:numE1]=(bXXTE1)
    moment2[numE1:NE,numE1:NE]=(bXXTE2)
    moment2[NE:,NE:]=bXXTII
    print(np.shape(moment2))
    B2=np.squeeze(C*(nAm.T@(moment2)@mAm)*gaverage**2)
    lambda_test2=np.roots(coeff2)
    roots = Cubic([1,-eigvAm[0],0,-B2])
    coeffB4[idxeta]=-B2
    envelopeRe[idxeta]=np.mean(-np.abs(np.squeeze(Beigvseries[idxeta,:,2])))
    lambda_theo2[idxeta,:]=roots
    print(lambda_test2[0])


# fig,ax=plt.subplots(figsize=(5,3))  
for i in range(3):
    ax.plot(etaseries,lambda_theo2[:,i],color='red',linewidth=1.5,label='theo')
ax.plot(etaseries,envelopeRe,color='gray',linewidth=1.5,linestyle='--',label='theo')
# # fig,ax=plt.subplots(figsize=(5,3))  
# # ax.plot(coeffB4,color=cm[i],linewidth=1.5,label='theo')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
# # ax.legend()

2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
2.0
(1000, 1000)
(-2.85070392901203-2.220446049250313e-16j)
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
D:\UseApp\Anaconda3\

* overall perturbation -- $\Delta_g + \Delta_{\rho}$

In [10]:
'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry
### >>>>>>>> Matrices Calculation >>>>>>>>>>>>>
### Pi_{(1-\rho)\rho} and Pi_{\rho}
'''
DeltaL_theo,DeltaR_theo,DeltaL_num,DeltaR_num=np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex) 
DeltaLR_math,DeltaLR_num = np.zeros((neta,4),dtype=complex),np.zeros((neta,ntrial,4),dtype=complex)
Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)
## initial vectors LEFT & RIGHT same for identical trial
numE1=int(ppercentEsub[0]*NE)
for itrial in range(ntrial):
    # initial eigenvectors
    Leigvecinit[itrial,:],Reigvecinit[itrial,:]=np.squeeze(Leigvecseries[0,itrial,:]).copy(),np.squeeze(Reigvecseries[0,itrial,:]).copy()
    l0,r0=np.reshape(yAm,(2*NE,1)),np.reshape(xAm,(2*NE,1))

    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        eta = etaseries[idxeta]
        etaE1,etaE12B,etaE2=eta*coeffetaEsub[0],eta*coeffetaEsub[1],eta*coeffetaEsub[2]
        etaE1IB,etaE2IB,etaI=eta*coeffetaTotal[0],eta*coeffetaTotal[1],eta*coeffetaTotal[2]
        rhoE1,rhoE12B,rhoE2= (1-np.sqrt(1-etaE1**2))/2.0,(1-np.sqrt(1-etaE12B**2))/2.0,(1-np.sqrt(1-etaE2**2))/2.0
        rhoE1IB,rhoE2IB,rhoI= (1-np.sqrt(1-etaE1IB**2))/2.0,(1-np.sqrt(1-etaE2IB**2))/2.0,(1-np.sqrt(1-etaI**2))/2.0

        # # numerical value
        # changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:]),(NE*2,1))-np.reshape(yAm,(2*NE,1))#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        # changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:]),(NE*2,1))-np.reshape(xAm,(2*NE,1))#r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))

        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:].copy()),(NE*2,1))#-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        changeL_num[:numE1,0]-=np.mean(changeL_num[:numE1,0])
        changeL_num[numE1:NE,0]-=np.mean(changeL_num[numE1:NE,0])
        changeL_num[NE:,0]-=np.mean(changeL_num[NE:,0])
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:].copy()),(NE*2,1))#-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        changeR_num[:numE1,0]-=np.mean(changeR_num[:numE1,0])
        changeR_num[numE1:NE,0]-=np.mean(changeR_num[numE1:NE,0])
        changeR_num[NE:,0]-=np.mean(changeR_num[NE:,0])

        DeltaLR_num[idxeta,itrial,3]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:numE1,0]*changeR_num[:numE1,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[numE1:NE,0]*changeR_num[numE1:NE,0])
        DeltaLR_num[idxeta,itrial,2]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])

        
        PIrho,PIrhominusrho=np.zeros((NE*2,NE*2)),np.zeros((NE*2,NE*2))
        PIrhoE1C,PIrhominusrhoE1C=np.zeros((NE*2,NE*2)),np.zeros((NE*2,NE*2))
        PIrhoE2C,PIrhominusrhoE2C=np.zeros((NE*2,NE*2)),np.zeros((NE*2,NE*2))
        PIrhoIC,PIrhominusrhoIC=np.zeros((NE*2,NE*2)),np.zeros((NE*2,NE*2))
        ### >>>>>>>>>> E1E1 BLOCK >>>>>>>>>
        ### >>> E1 contribute >>>>
        hNE = int(NE*ppercentEsub[0])
        PIrho[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercentEsub[0])*(np.sqrt(rhoE1)+0)/NE/2.0*signetaEsub[0]
        PIrhominusrho[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercentEsub[0])*(np.sqrt((1-rhoE1)*rhoE1)+0)/NE/2.0*signetaEsub[0]
        PIrhoE1C[:hNE,:hNE]=np.eye(hNE)*(NE*ppercentEsub[0])*(np.sqrt(rhoE1)+0)/NE/2.0*signetaEsub[0]
        PIrhominusrhoE1C[:hNE,:hNE]=np.eye(hNE)*(NE*ppercentEsub[0])*(np.sqrt((1-rhoE1)*rhoE1)+0)/NE/2.0*signetaEsub[0]
        ### >>> E2 contribute >>>>
        PIrho[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercentEsub[1])*(np.sqrt(rhoE12B)+0)/NE/2.0*signetaEsub[1]
        PIrhominusrho[:hNE,:hNE]+=np.eye(hNE)*(NE*ppercentEsub[1])*(np.sqrt((1-rhoE12B)*rhoE12B)+0)/NE/2.0*signetaEsub[1]
        PIrhoE2C[:hNE,:hNE]=np.eye(hNE)*(NE*ppercentEsub[1])*(np.sqrt(rhoE12B)+0)/NE/2.0*signetaEsub[1]
        PIrhominusrhoE2C[:hNE,:hNE]=np.eye(hNE)*(NE*ppercentEsub[1])*(np.sqrt((1-rhoE12B)*rhoE12B)+0)/NE/2.0*signetaEsub[1]
        ### >>> I contribute >>>>
        PIrho[:hNE,:hNE]+=np.eye(hNE)*(NE*1.0)*(np.sqrt(rhoE1IB)+0)/NE/2.0*signetaTotal[0]
        PIrhominusrho[:hNE,:hNE]+=np.eye(hNE)*(NE*1.0)*(np.sqrt((1-rhoE1IB)*rhoE1IB)+0)/NE/2.0*signetaTotal[0]
        PIrhoIC[:hNE,:hNE]=np.eye(hNE)*(NE*1.0)*(np.sqrt(rhoE1IB)+0)/NE/2.0*signetaTotal[0]
        PIrhominusrhoIC[:hNE,:hNE]=np.eye(hNE)*(NE*1.0)*(np.sqrt((1-rhoE1IB)*rhoE1IB)+0)/NE/2.0*signetaTotal[0]
    
        ### >>>>>>>>>> E2E2 BLOCK >>>>>>>>>
        ### >>> E1 contribute >>>>
        PIrho[hNE:NE,hNE:NE]+=np.eye(NE-hNE)*(NE*ppercentEsub[0])*(np.sqrt(rhoE12B)+0)/NE/2.0*signetaEsub[1]
        PIrhominusrho[hNE:NE,hNE:NE]+=np.eye(NE-hNE)*(NE*ppercentEsub[0])*(np.sqrt((1-rhoE12B)*rhoE12B)+0)/NE/2.0*signetaEsub[1]
        PIrhoE1C[hNE:NE,hNE:NE]=np.eye(NE-hNE)*(NE*ppercentEsub[0])*(np.sqrt(rhoE12B)+0)/NE/2.0*signetaEsub[1]
        PIrhominusrhoE1C[hNE:NE,hNE:NE]=np.eye(NE-hNE)*(NE*ppercentEsub[0])*(np.sqrt((1-rhoE12B)*rhoE12B)+0)/NE/2.0*signetaEsub[1]
        ### >>> E2 contribute >>>>
        PIrho[hNE:NE,hNE:NE]+=np.eye(NE-hNE)*(NE*ppercentEsub[1])*(np.sqrt(rhoE2)+0)/NE/2.0*signetaEsub[2]
        PIrhominusrho[hNE:NE,hNE:NE]+=np.eye(NE-hNE)*(NE*ppercentEsub[1])*(np.sqrt((1-rhoE2)*rhoE2)+0)/NE/2.0*signetaEsub[2]
        PIrhoE2C[hNE:NE,hNE:NE]=np.eye(NE-hNE)*(NE*ppercentEsub[1])*(np.sqrt(rhoE2)+0)/NE/2.0*signetaEsub[2]
        PIrhominusrhoE2C[hNE:NE,hNE:NE]=np.eye(NE-hNE)*(NE*ppercentEsub[1])*(np.sqrt((1-rhoE2)*rhoE2)+0)/NE/2.0*signetaEsub[2]
        ### >>> I contribute >>>>
        PIrho[hNE:NE,hNE:NE]+=np.eye(NE-hNE)*(NE*1.0)*(np.sqrt(rhoE2IB)+0)/NE/2.0*signetaTotal[1]
        PIrhominusrho[hNE:NE,hNE:NE]+=np.eye(NE-hNE)*(NE*1.0)*(np.sqrt((1-rhoE2IB)*rhoE2IB)+0)/NE/2.0*signetaTotal[1]
        PIrhoIC[hNE:NE,hNE:NE]=np.eye(NE-hNE)*(NE*1.0)*(np.sqrt(rhoE2IB)+0)/NE/2.0*signetaTotal[1]
        PIrhominusrhoIC[hNE:NE,hNE:NE]=np.eye(NE-hNE)*(NE*1.0)*(np.sqrt((1-rhoE2IB)*rhoE2IB)+0)/NE/2.0*signetaTotal[1]

        ### >>>>>>>>>> II BLOCK >>>>>>>>>
        ### >>> E1 contribute >>>>
        PIrho[NE:,NE:]+=np.eye(NE)*(NE*ppercentEsub[0])*(np.sqrt(rhoE1IB)+0)/NE/2.0*signetaTotal[0]
        PIrhominusrho[NE:,NE:]+=np.eye(NE)*(NE*ppercentEsub[0])*(np.sqrt((1-rhoE1IB)*rhoE1IB)+0)/NE/2.0*signetaTotal[0]
        PIrhoE1C[NE:,NE:]=np.eye(NE)*(NE*ppercentEsub[0])*(np.sqrt(rhoE1IB)+0)/NE/2.0*signetaTotal[0]
        PIrhominusrhoE1C[NE:,NE:]=np.eye(NE)*(NE*ppercentEsub[0])*(np.sqrt((1-rhoE1IB)*rhoE1IB)+0)/NE/2.0*signetaTotal[0]
        ### >>> E2 contribute >>>>
        PIrho[NE:,NE:]+=np.eye(NE)*(NE*ppercentEsub[1])*(np.sqrt(rhoE2IB)+0)/NE/2.0*signetaTotal[1]
        PIrhominusrho[NE:,NE:]+=np.eye(NE)*(NE*ppercentEsub[1])*(np.sqrt((1-rhoE2IB)*rhoE2IB)+0)/NE/2.0*signetaTotal[1]
        PIrhoE2C[NE:,NE:]=np.eye(NE)*(NE*ppercentEsub[1])*(np.sqrt(rhoE2IB)+0)/NE/2.0*signetaTotal[1]
        PIrhominusrhoE2C[NE:,NE:]=np.eye(NE)*(NE*ppercentEsub[1])*(np.sqrt((1-rhoE2IB)*rhoE2IB)+0)/NE/2.0*signetaTotal[1]
        ### >>> I contribute >>>>
        PIrho[NE:,NE:]+=np.eye(NE)*(NE*1.0)*(np.sqrt(rhoI)+0)/NE/2.0*signetaTotal[2]
        PIrhominusrho[NE:,NE:]+=np.eye(NE)*(NE*1.0)*(np.sqrt((1-rhoI)*rhoI)+0)/NE/2.0*signetaTotal[2]
        PIrhoIC[NE:,NE:]=np.eye(NE)*(NE*1.0)*(np.sqrt(rhoI)+0)/NE/2.0*signetaTotal[2]
        PIrhominusrhoIC[NE:,NE:]=np.eye(NE)*(NE*1.0)*(np.sqrt((1-rhoI)*rhoI)+0)/NE/2.0*signetaTotal[2]

        DeltaLR_math[idxeta,3]=np.reshape(l0,(1,NE*2))@(2*PIrhominusrho)@np.reshape(r0,(2*NE,1))*gaverage**2/eigvAm[0]**2
        # Excitatory
        DeltaLR_math[idxeta,0]=np.reshape(l0,(1,NE*2))@(2*PIrhominusrhoE1C)@np.reshape(r0,(2*NE,1))*gaverage**2/eigvAm[0]**2

        DeltaLR_math[idxeta,1]=np.reshape(l0,(1,NE*2))@(2*PIrhominusrhoE2C)@np.reshape(r0,(2*NE,1))*gaverage**2/eigvAm[0]**2
        DeltaLR_math[idxeta,2]=np.reshape(l0,(1,NE*2))@(2*PIrhominusrhoIC)@np.reshape(r0,(2*NE,1))*gaverage**2/eigvAm[0]**2
        #(2*aampT*aampO*gaverage**2-2*aampT*gaverage**2)/eigvAm[0]**2
        # if idxeta==neta-1:
        #     DeltaLR_math[idxeta,0]=-DeltaLR_math[idxeta,0]
        #     DeltaLR_math[idxeta,1]=-DeltaLR_math[idxeta,1]
        # DeltaLR_math[idxeta,2]=DeltaLR_math[idxeta,0]+DeltaLR_math[idxeta,1]


In [11]:
'''
theoretical overlap, we calculate the value use mathematical equations
'''
'''
Total Theoretical Solution for Covariance
'''

'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry
'''
DeltaLR_matheta,DeltaLR_mathg,DeltaLR_num = np.zeros((neta,4),dtype=complex),np.zeros((neta,4),dtype=complex),np.zeros((neta,ntrial,4),dtype=complex)

Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)
## initial vectors LEFT & RIGHT same for identical trial
numE1= int(NE*ppercentEsub[0])
numE2=NE-numE1
for itrial in range(ntrial):
    # initial eigenvectors
    l0,r0=np.reshape(Leigvecseries[0,itrial,:].copy(),(2*NE,1)),np.reshape(Reigvecseries[0,itrial,:].copy(),(2*NE,1))#np.reshape(-yAm,(2*NE,1)),np.reshape(-xAm,(2*NE,1))#

    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        eta = etaseries[idxeta]
        etaE1,etaE12B,etaE2,etaE1IB,etaE2IB,etaI=eta*coeffetaEsub[0],eta*coeffetaEsub[1],eta*coeffetaEsub[2],eta*coeffetaTotal[0],eta*coeffetaTotal[1],eta*coeffetaTotal[2]
        rhoE1,rhoE12B,rhoE2= (1-np.sqrt(1-etaE1**2))/2.0,(1-np.sqrt(1-etaE12B**2))/2.0,(1-np.sqrt(1-etaE2**2))/2.0
        rhoE1IB,rhoE2IB,rhoI= (1-np.sqrt(1-etaE1IB**2))/2.0,(1-np.sqrt(1-etaE2IB**2))/2.0,(1-np.sqrt(1-etaI**2))/2.0
        print('etaset:',etaE1,etaE12B,etaE2,etaE1IB,etaE2IB,etaI)

        print('rhoset:',rhoE1,rhoE12B,rhoE2,rhoE1IB,rhoE2IB,rhoI)

        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:].copy()),(NE*2,1))-np.reshape(yAm,(2*NE,1))#-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        # changeL_num[:numE1,0]-=np.mean(changeL_num[:numE1,0])
        # changeL_num[numE1:NE,0]-=np.mean(changeL_num[numE1:NE,0])
        # changeL_num[NE:,0]-=np.mean(changeL_num[NE:,0])
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:].copy()),(NE*2,1))-np.reshape(xAm,(2*NE,1))#-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        # changeR_num[:numE1,0]-=np.mean(changeR_num[:numE1,0])
        # changeR_num[numE1:NE,0]-=np.mean(changeR_num[numE1:NE,0])
        # changeR_num[NE:,0]-=np.mean(changeR_num[NE:,0])

        DeltaLR_num[idxeta,itrial,3]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:numE1,0]*changeR_num[:numE1,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[numE1:NE,0]*changeR_num[numE1:NE,0])
        DeltaLR_num[idxeta,itrial,2]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])
   
        # EXCITATORY
        # JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE)    
        # -JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)        bXXTE1,bXXTE2,bXXTII=np.eye(numE1),np.eye(numE2),np.eye(NE)

        '''
        ### >>>>>>>>>>> delta in eta
        '''       
        DeltaLR_matheta[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]**2)*(etaE1-2.0*np.sqrt(rhoE1))*signetaEsub[0]/4.0
        DeltaLR_matheta[idxeta,0]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]*ppercentEsub[1])*(etaE12B-2.0*np.sqrt(rhoE12B))*signetaEsub[1]/4.0 
        DeltaLR_matheta[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]*1.0)*(etaE1IB-2.0*np.sqrt(rhoE1IB))*signetaTotal[0]/4.0)
        print('DeltaE1:',DeltaLR_matheta[idxeta,0])

        DeltaLR_matheta[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]**2)*(etaE2-2.0*np.sqrt(rhoE2))*signetaEsub[2]/4.0 
        DeltaLR_matheta[idxeta,1]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]*ppercentEsub[0])*(etaE12B-2.0*np.sqrt(rhoE12B))*signetaEsub[1]/4.0 
        DeltaLR_matheta[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]*1.0)*(etaE2IB-2.0*np.sqrt(rhoE2IB))*signetaTotal[1]/4.0)
        print('DeltaE2:',DeltaLR_matheta[idxeta,1])

        DeltaLR_matheta[idxeta,2]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*ppercentEsub[0])*(etaE1IB-2.0*np.sqrt(rhoE1IB))*signetaTotal[0]/4.0 
        DeltaLR_matheta[idxeta,2]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*ppercentEsub[1])*(etaE2IB-2.0*np.sqrt(rhoE2IB))*signetaTotal[1]/4.0 
        DeltaLR_matheta[idxeta,2]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*1.0)*(etaI-2.0*np.sqrt(rhoI))*signetaTotal[2]/4.0)
        '''
        ### >>>>>>>>>>> delta in g with eta
        '''  
        DeltaLR_mathg[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]**2)*(np.sqrt(rhoE1))*signetaEsub[0]/2.0
        DeltaLR_mathg[idxeta,0]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]*ppercentEsub[1])*(np.sqrt(rhoE12B))*signetaEsub[1]/2.0 
        DeltaLR_mathg[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]*1.0)*(np.sqrt(rhoE1IB))*signetaTotal[0]/2.0)
        print('DeltaE1_g:',DeltaLR_mathg[idxeta,0])

        DeltaLR_mathg[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]**2)*(np.sqrt(rhoE2))*signetaEsub[2]/2.0 
        DeltaLR_mathg[idxeta,1]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]*ppercentEsub[0])*(np.sqrt(rhoE12B))*signetaEsub[1]/2.0 
        DeltaLR_mathg[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]*1.0)*(np.sqrt(rhoE2IB))*signetaTotal[1]/2.0)
        print('DeltaE2:',DeltaLR_mathg[idxeta,1])

        DeltaLR_mathg[idxeta,2]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*ppercentEsub[0])*(np.sqrt(rhoE1IB))*signetaTotal[0]/2.0 
        DeltaLR_mathg[idxeta,2]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*ppercentEsub[1])*(np.sqrt(rhoE2IB))*signetaTotal[1]/2.0 
        DeltaLR_mathg[idxeta,2]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*1.0)*(np.sqrt(rhoI))*signetaTotal[2]/2.0)

        DeltaLR_matheta[idxeta,3]=2*DeltaLR_matheta[idxeta,2]+2*DeltaLR_matheta[idxeta,1]+2*DeltaLR_matheta[idxeta,0]
        DeltaLR_mathg[idxeta,3]=2*DeltaLR_mathg[idxeta,2]+2*DeltaLR_mathg[idxeta,1]+2*DeltaLR_mathg[idxeta,0]

09999999999999998 0.5 0.5 0.5
DeltaE1: (-0.01146815761831182+0j)
DeltaE2: (-0.01146815761831182+0j)
DeltaE1_g: (0.03706815761831151+0j)
DeltaE2: (0.03706815761831151+0j)
etaset: 0.0 0.0 0.0 0.0 0.0 0.0
rhoset: 0.0 0.0 0.0 0.0 0.0 0.0
DeltaE1: 0j
DeltaE2: 0j
DeltaE1_g: 0j
DeltaE2: 0j
etaset: 0.06666666666666667 0.0 0.06666666666666667 0.1111111111111111 0.1111111111111111 0.1111111111111111
rhoset: 0.0011123484301411302 0.0 0.0011123484301411302 0.003096005000046753 0.003096005000046753 0.003096005000046753
DeltaE1: (-4.678782525272427e-06+0j)
DeltaE2: (-4.678782525272427e-06+0j)
DeltaE1_g: (0.0028491232269696816+0j)
DeltaE2: (0.0028491232269696816+0j)
etaset: 0.13333333333333333 0.0 0.13333333333333333 0.2222222222222222 0.2222222222222222 0.2222222222222222
rhoset: 0.004464375089383132 0.0 0.004464375089383132 0.012501978478215436 0.012501978478215436 0.012501978478215436
DeltaE1: (-3.8069083451365146e-05+0j)
DeltaE2: (-3.8069083451365146e-05+0j)
DeltaE1_g: (0.0057269579723401845+0j)


In [12]:
itrial=3
fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
ax.plot(etaseries,DeltaLR_matheta[:,3]+DeltaLR_mathg[:,3],color=cl[1],linewidth=1.5)
ax.plot(etaseries,DeltaLR_math[:,3],color=cl[0],linewidth=1.5)
ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_num[:,:,3]),axis=1),color=cl[2],linewidth=1.5,linestyle='--')

D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
